<center>
<img src="https://tensorflowkorea.files.wordpress.com/2020/12/4.-e18492e185a9e186abe1848ce185a1-e18480e185a9e186bce18487e185aee18492e185a1e18482e185b3e186ab-e18486e185a5e18489e185b5e186abe18485e185a5e18482e185b5e186bce18483e185b5e186b8e18485e185a5e.png?w=972" width="200" height="200"><br>
</center>

## 06-3 주성분 분석

### - 차원과 차원 축소

지금까지 우리는 데이터가 가진 속성을 특성이라 불렀다. 앞서 과일 사진의 경우 10000개의 픽셀이 있기 때문에 10000개의 특성이 있는 것이다. 머신러닝에서는 이런 특성을 차원이라고 부른다. 10000개의 특성은 결국 10000개의 차원이라는 것인데 이 차원을 줄일 수 있다면 저장 공간을 크게 절약할 수 있다.

이를 위해 비지도 학습 작업 중 하나인 차원 축소 알고리즘을 다뤄보자. 차원 축소는 데이터를 가장 잘 나타내는 일부 특성을 선택하여 데이터 크기를 줄이고 지도 학습 모델의 성능을 향상시킬 수 있는 방법이다.

또한 줄어든 차원에서 다시 원본 차원으로 손실을 최대한 줄이면서 복원할 수도 있다. 이번엔 대표적인 차원 축소 알고리즘인 주성분 분석을 배우자. 주성분 분석은 간단히 PCA라고 부른다.

### - 주성분 분석 소개

주성분 분석은 데이터에 있는 분산이 큰 방향을 찾는 것으로 이해할 수 있다. 분산은 데이터가 널리 퍼져있는 정도를 말한다. 분산이 큰 방향을 데이터로 잘 표현하는 벡터로 생각할 수 있다.